In [1]:
import numpy as np
import tensornetwork as tn
import tensorflow as tf
import numba
import json

# Layers

In [34]:
class QuantumDMRGLayer(tf.keras.layers.Layer):
    def __init__(self, dimvec, pos_label, nblabels, bond_len, nearzero_std=1e-9, isolated_labelnode=True):
        super(QuantumDMRGLayer, self).__init__()
        self.dimvec = dimvec
        self.pos_label = pos_label
        self.nblabels = nblabels
        self.m = bond_len
        self.isolated_label = isolated_labelnode

        assert self.pos_label >= 0 and self.pos_label < self.dimvec

        self.mps_tensors = [tf.Variable(self.mps_tensor_initial_values(i, nearzero_std=nearzero_std),
                                        trainable=True,
                                        name='mps_tensors_{}'.format(i))
                            for i in range(self.dimvec)]
        if self.isolated_label:
            self.output_tensor = tf.Variable(tf.random.normal((self.m, self.m, self.nblabels),
                                                              mean=0.0,
                                                              stddev=nearzero_std),
                                             trainable=True,
                                             name='mps_output_node')

    def mps_tensor_initial_values(self, idx, nearzero_std=1e-9):
        if idx == 0 or idx == self.dimvec - 1:
            tempmat = tf.eye(max(2, self.m))
            mat = tempmat[0:2, :] if 2 < self.m else tempmat[:, 0:self.m]
            return mat + tf.random.normal(mat.shape, mean=0.0, stddev=nearzero_std)
        elif not self.isolated_label and idx == self.pos_label:
            return tf.random.normal((2, self.m, self.m, self.nblabels),
                                    mean=0.0,
                                    stddev=nearzero_std)
        else:
            return tf.random.normal((2, self.m, self.m),
                                    mean=0.0,
                                    stddev=nearzero_std)

    def infer_single(self, input):
        assert input.shape[0] == self.dimvec
        assert input.shape[1] == 2

        nodes = [
            tn.Node(self.mps_tensors[i], backend='tensorflow')
            for i in range(self.dimvec)
        ]
        if self.isolated_label:
            output_node = tn.Node(self.output_tensor, backend='tensorflow')
        input_nodes = [
            tn.Node(input[i, :], backend='tensorflow')
            for i in range(self.dimvec)
        ]

        for i in range(self.dimvec):
            nodes[i][0] ^ input_nodes[i][0]
        if self.isolated_label:
            nodes[0][1] ^ nodes[1][1]
            for i in range(1, self.pos_label):
                nodes[i][2] ^ nodes[i + 1][1]
            nodes[self.pos_label][2] ^ output_node[0]
            output_node[1] ^ nodes[self.pos_label + 1][1]
            for i in range(self.pos_label + 1, self.dimvec - 1):
                nodes[i][2] ^ nodes[i + 1][1]
        else:
            nodes[0][1] ^ nodes[1][1]
            for i in range(1, self.dimvec-1):
                nodes[i][2] ^ nodes[i + 1][1]

        if self.isolated_label:
            final_node = tn.contractors.auto(nodes + input_nodes + [output_node],
                                             output_edge_order=[output_node[2]])
        else:
            final_node = tn.contractors.auto(nodes + input_nodes,
                                             output_edge_order=[nodes[self.pos_label][3]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single, inputs)

# Generating Data

In [10]:
def generate_random_data(size, std=0.2):
    centroids = {0: np.array([0.5, 0.5, 0, 1]), 
                 1: np.array([-0.5, 0.5, 0, -1]),
                 2: np.array([-0.5, -0.5, 1, 0]),
                 3: np.array([0.5, -0.5, -1, 0])}
    for _ in range(size):
        quadrant = np.random.choice(range(4))
        yield np.random.normal(loc=centroids[quadrant], scale=std), quadrant
        

@numba.njit(numba.float64[:, :](numba.float64[:]))
def convert_pixels_to_tnvector(vector):
    tnvector = np.concatenate((
        np.array([[vector[0], np.sign(vector[0])*(1-np.abs(vector[0]))]]),
        np.array([[vector[1], np.sign(vector[1])*(1-np.abs(vector[1]))]]),
        np.array([[vector[2], np.sign(vector[2])*(1-np.abs(vector[0]))]]),
        np.array([[vector[3], np.sign(vector[1])*(1-np.abs(vector[1]))]])

    ), axis=0)
    return tnvector


In [11]:
size = 1000

X = np.zeros((size, 4, 2))
Y = np.zeros((size, 4))

for i, (thisX, thisY) in enumerate(generate_random_data(size)):
    X[i, :, :] = convert_pixels_to_tnvector(thisX)
    Y[i, thisY] = 1.

# Model

In [12]:
def QuantumKerasModel(dimvec, pos_label, nblabels, bond_len, nearzero_std=1e-9, optimizer='adam'):
    quantum_dmrg_model = tf.keras.Sequential([
        tf.keras.Input(shape=(dimvec, 2)),
        QuantumDMRGLayer(dimvec=dimvec,
                         pos_label=pos_label,
                         nblabels=nblabels,
                         bond_len=bond_len,
                         nearzero_std=nearzero_std),
#         tf.keras.layers.LayerNormalization(beta_initializer='RandomUniform', gamma_initializer='RandomUniform', beta_constraint='non_neg')
        tf.keras.layers.LayerNormalization(beta_initializer='RandomUniform', gamma_initializer='RandomUniform'),
        tf.keras.layers.Softmax()
    ])
    quantum_dmrg_model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
    return quantum_dmrg_model

In [58]:
 # model parameters
dimvec = 4
pos_label = 2
nblabels = 4
bond_len = 2
nbdata = 1000

# training and CV parameters
nb_epochs = 1000
cv_fold = 5
batch_size = 10
std = 1e-9
learning_rate = 1e-2

In [18]:
# Prepare for cross-validation
cv_labels = np.random.choice(range(cv_fold), size=nbdata)

In [19]:
cv_idx = 0

trainX = X[cv_labels==cv_idx, :, :]
trainY = Y[cv_labels==cv_idx, :]

In [59]:
quantum_dmrg_model = QuantumKerasModel(dimvec=dimvec, pos_label=pos_label, nblabels=nblabels, bond_len=bond_len,
                                       nearzero_std=std, optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

In [60]:
quantum_dmrg_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantum_dmrg_layer_7 (Quantu (None, 4)                 40        
_________________________________________________________________
layer_normalization_7 (Layer (None, 4)                 8         
_________________________________________________________________
softmax_7 (Softmax)          (None, 4)                 0         
Total params: 48
Trainable params: 48
Non-trainable params: 0
_________________________________________________________________


In [61]:
tn_layer = quantum_dmrg_model.layers[0]
tn_layer.trainable_variables[0:3]

[<tf.Variable 'mps_tensors_0:0' shape=(2, 2) dtype=float32, numpy=
 array([[ 1.0000000e+00, -2.4472582e-10],
        [-3.1551026e-10,  1.0000000e+00]], dtype=float32)>,
 <tf.Variable 'mps_tensors_1:0' shape=(2, 2, 2) dtype=float32, numpy=
 array([[[-1.7868844e-09,  1.5684126e-10],
         [-4.5113749e-10,  8.0341755e-10]],
 
        [[ 2.9506839e-10, -1.2818342e-09],
         [ 1.5821333e-09, -5.1799121e-10]]], dtype=float32)>,
 <tf.Variable 'mps_tensors_2:0' shape=(2, 2, 2) dtype=float32, numpy=
 array([[[-1.2069160e-09, -7.2580462e-11],
         [ 1.1222002e-09, -2.5530027e-09]],
 
        [[ 8.7495028e-10,  9.6011843e-10],
         [ 8.5000867e-10,  2.0877959e-09]]], dtype=float32)>]

In [62]:
X.shape

(1000, 4, 2)

In [63]:
output = quantum_dmrg_model.predict(X[0:, :, :])
output

array([[0.25478882, 0.24265279, 0.25557625, 0.24698208],
       [0.25478882, 0.24265279, 0.25557625, 0.24698208],
       [0.25478882, 0.24265279, 0.25557625, 0.24698208],
       ...,
       [0.25478882, 0.24265279, 0.25557625, 0.24698208],
       [0.25478882, 0.24265279, 0.25557625, 0.24698208],
       [0.25478882, 0.24265279, 0.25557625, 0.24698208]], dtype=float32)

In [64]:
quantum_dmrg_model.fit(trainX, trainY, epochs=nb_epochs, batch_size=batch_size)

Epoch 1/1000
21/21 [==============================] - 0s 15ms/step - loss: 1.3878: 0s - loss: 1.38
Epoch 2/1000
21/21 [==============================] - 0s 14ms/step - loss: 1.3837
Epoch 3/1000
21/21 [==============================] - 0s 10ms/step - loss: 1.3835
Epoch 4/1000
21/21 [==============================] - 0s 6ms/step - loss: 1.3831
Epoch 5/1000
21/21 [==============================] - 0s 13ms/step - loss: 1.3831
Epoch 6/1000
21/21 [==============================] - 0s 4ms/step - loss: 1.3827
Epoch 7/1000
21/21 [==============================] - 0s 6ms/step - loss: 1.3832
Epoch 8/1000
21/21 [==============================] - 0s 10ms/step - loss: 1.3829
Epoch 9/1000
21/21 [==============================] - 0s 11ms/step - loss: 1.3839
Epoch 10/1000
21/21 [==============================] - 0s 5ms/step - loss: 1.3839
Epoch 11/1000
21/21 [==============================] - 0s 9ms/step - loss: 1.3825
Epoch 12/1000
21/21 [==============================] - 0s 10ms/step - loss: 1.3825
E

W0628 20:05:07.647438 4521258432 callbacks.py:307] Method (on_train_batch_end) is slow compared to the batch update (0.101582). Check your callbacks.


21/21 [==============================] - 0s 13ms/step - loss: 1.3833
Epoch 22/1000
21/21 [==============================] - 0s 10ms/step - loss: 1.3830
Epoch 23/1000
21/21 [==============================] - 0s 5ms/step - loss: 1.3838
Epoch 24/1000
21/21 [==============================] - 0s 18ms/step - loss: 1.3824
Epoch 25/1000
21/21 [==============================] - 0s 4ms/step - loss: 1.3833
Epoch 26/1000
21/21 [==============================] - 0s 5ms/step - loss: 1.3830
Epoch 27/1000
21/21 [==============================] - 0s 6ms/step - loss: 1.3831
Epoch 28/1000
21/21 [==============================] - 0s 10ms/step - loss: 1.3823
Epoch 29/1000
21/21 [==============================] - 0s 6ms/step - loss: 1.3825
Epoch 30/1000
21/21 [==============================] - 0s 11ms/step - loss: 1.3828
Epoch 31/1000
21/21 [==============================] - 0s 8ms/step - loss: 1.3832
Epoch 32/1000
21/21 [==============================] - 1s 32ms/step - loss: 1.3836
Epoch 33/1000
21/21 [===

W0628 20:05:16.056790 4521258432 callbacks.py:307] Method (on_train_batch_end) is slow compared to the batch update (0.170837). Check your callbacks.


21/21 [==============================] - 1s 27ms/step - loss: 1.3827
Epoch 47/1000
21/21 [==============================] - 0s 14ms/step - loss: 1.3820
Epoch 48/1000
21/21 [==============================] - 0s 12ms/step - loss: 1.3824
Epoch 49/1000
21/21 [==============================] - 0s 18ms/step - loss: 1.3835
Epoch 50/1000
21/21 [==============================] - 0s 9ms/step - loss: 1.3829
Epoch 51/1000
21/21 [==============================] - 0s 12ms/step - loss: 1.3830
Epoch 52/1000
21/21 [==============================] - 0s 18ms/step - loss: 1.3841
Epoch 53/1000
21/21 [==============================] - 0s 5ms/step - loss: 1.3835
Epoch 54/1000
21/21 [==============================] - 0s 6ms/step - loss: 1.3836
Epoch 55/1000
21/21 [==============================] - 0s 9ms/step - loss: 1.3838
Epoch 56/1000
21/21 [==============================] - 0s 18ms/step - loss: 1.3858
Epoch 57/1000
21/21 [==============================] - 0s 13ms/step - loss: 1.3843
Epoch 58/1000
21/21 [=

...

21/21 [==============================] - 0s 20ms/step - loss: 1.3826
Epoch 917/1000
21/21 [==============================] - 0s 4ms/step - loss: 1.3821
Epoch 918/1000
21/21 [==============================] - 0s 12ms/step - loss: 1.3829
Epoch 919/1000
21/21 [==============================] - 0s 12ms/step - loss: 1.3819
Epoch 920/1000
21/21 [==============================] - 0s 11ms/step - loss: 1.3823
Epoch 921/1000
21/21 [==============================] - 0s 14ms/step - loss: 1.3827
Epoch 922/1000
21/21 [==============================] - 0s 12ms/step - loss: 1.3846
Epoch 923/1000
21/21 [==============================] - 0s 17ms/step - loss: 1.3846: 0s - loss: 1.3
Epoch 924/1000
21/21 [==============================] - 0s 13ms/step - loss: 1.3839
Epoch 925/1000
21/21 [==============================] - 0s 11ms/step - loss: 1.3822
Epoch 926/1000
21/21 [==============================] - 0s 5ms/step - loss: 1.3830
Epoch 927/1000
21/21 [==============================] - 0s 13ms/step - loss: 

W0628 20:09:56.115052 4521258432 callbacks.py:307] Method (on_train_batch_end) is slow compared to the batch update (0.128340). Check your callbacks.


21/21 [==============================] - 0s 6ms/step - loss: 1.3830
Epoch 968/1000
21/21 [==============================] - 0s 11ms/step - loss: 1.3822
Epoch 969/1000
21/21 [==============================] - 0s 7ms/step - loss: 1.3832
Epoch 970/1000
21/21 [==============================] - 0s 8ms/step - loss: 1.3839
Epoch 971/1000
 1/21 [>.............................] - ETA: 0s - loss: 1.3224

W0628 20:09:57.103764 4521258432 callbacks.py:307] Method (on_train_batch_end) is slow compared to the batch update (0.122230). Check your callbacks.


21/21 [==============================] - 0s 20ms/step - loss: 1.3834
Epoch 972/1000
21/21 [==============================] - 0s 9ms/step - loss: 1.3830
Epoch 973/1000
21/21 [==============================] - 0s 9ms/step - loss: 1.3833
Epoch 974/1000
21/21 [==============================] - 0s 12ms/step - loss: 1.3829: 0s - loss: 1.382
Epoch 975/1000
21/21 [==============================] - 0s 9ms/step - loss: 1.3830
Epoch 976/1000
21/21 [==============================] - 0s 20ms/step - loss: 1.3823
Epoch 977/1000
21/21 [==============================] - 0s 18ms/step - loss: 1.3827
Epoch 978/1000
21/21 [==============================] - 0s 4ms/step - loss: 1.3836
Epoch 979/1000
21/21 [==============================] - 0s 8ms/step - loss: 1.3841
Epoch 980/1000
21/21 [==============================] - 0s 8ms/step - loss: 1.3846
Epoch 981/1000
21/21 [==============================] - 0s 7ms/step - loss: 1.3828
Epoch 982/1000
21/21 [==============================] - 0s 15ms/step - loss: 1.3

In [65]:
output = quantum_dmrg_model.predict(X)
output

array([[0.24470682, 0.28424042, 0.24196967, 0.22908309],
       [0.24470682, 0.28424042, 0.24196967, 0.22908309],
       [0.24470682, 0.28424042, 0.24196967, 0.22908309],
       ...,
       [0.24470682, 0.28424042, 0.24196967, 0.22908309],
       [0.24470682, 0.28424042, 0.24196967, 0.22908309],
       [0.24470682, 0.28424042, 0.24196967, 0.22908309]], dtype=float32)

In [42]:
tn_layer.trainable_variables[0:3]

[<tf.Variable 'mps_tensors_0:0' shape=(2, 2) dtype=float32, numpy=
 array([[ 1.0000000e+00, -4.8281468e-10],
        [ 5.7440247e-10,  1.0000000e+00]], dtype=float32)>,
 <tf.Variable 'mps_tensors_1:0' shape=(2, 2, 2) dtype=float32, numpy=
 array([[[ 1.01076369e-09, -9.79604731e-10],
         [ 1.82769536e-10,  1.14186494e-10]],
 
        [[-4.66370109e-10, -1.33745948e-09],
         [-1.02995341e-10, -5.84964133e-10]]], dtype=float32)>,
 <tf.Variable 'mps_tensors_2:0' shape=(2, 2, 2) dtype=float32, numpy=
 array([[[-1.3092647e-09,  1.6047685e-09],
         [-5.3281740e-10, -3.0815253e-09]],
 
        [[ 4.4603912e-10,  1.9199056e-09],
         [ 1.7281526e-09,  5.0067967e-11]]], dtype=float32)>]